# **Homework 7: Iterative Methods for Linear Algebra**

Read sections 4.4 and 4.7, and answer the following five problems. [Problems modified from the [online text](https://numericalmethodssullivan.github.io/ch-linearalgebra.html#applying-what-youve-learned-3)]

For all problems Python code is expected; handwritten work is not required.

In [2]:
import numpy as np
from tabulate import tabulate
from time import time
from scipy.linalg import lu_factor, lu_solve, lu

# Python functions for iterative methods

In [3]:
def jacobi(A, b, x0, numIter):
  # Inputs: A - numpy array (nxn)
  #         b - numpy array (nx1)
  #         x0 - initial vector (nx1)
  #         numIter - number of iterations (scalar)
  xk = x0.copy()
  for i in range (numIter):
    xprev = xk
    for j in range(len(xk)):
      LplusD_row = A[j,:].copy()
      LplusD_row[j] = 0
      xk[j] = ( b[j] - np.dot(LplusD_row,xprev) )/A[j,j]
  return xk

def gauss_seidel(A, b, x0, numIter):
  # Inputs: A - numpy array (nxn)
  #         b - numpy array (nx1)
  #         x0 - initial vector (nx1)
  #         numIter - number of iterations (scalar)
  xk = x0.copy()
  L = np.tril(A) - np.diag(np.diag(A))
  U = np.triu(A) - np.diag(np.diag(A))
  n = len(xk)
  for k in range(numIter):
    xprev = xk
    for j in range(n):
      xk[j] = ( b[j] - np.dot(U[j,:],xprev) - np.dot(L[j,:],xk))/A[j,j]
  return xk

def SOR(A, b, x0, omega, numIter):
  # Inputs: A - numpy array (nxn)
  #         b - numpy array (nx1)
  #         x0 - initial vector (nx1)
  #         numIter - number of iterations (scalar)
  xk = x0.copy()
  L = np.tril(A) - np.diag(np.diag(A))
  U = np.triu(A) - np.diag(np.diag(A))
  D = np.diag(np.diag(A))
  inv = np.linalg.inv(omega*L+D)
  for i in range(numIter):
    xprev = xk
    xk = inv @ ( ((1-omega)*D-omega*U) @ xk) + omega * inv @ b
  return xk

# Problem 1
*Exercise 4.91: Application of the power method on a stochastic matrix -- Markov chains*

(This concept for this problem is modified from [6]. The data is taken from NOAA and the National Weather Service with the specific values associated with La Crosse, WI.)

Floods in the Mississippi River Valleys of the upper midwest have somewhat predictable day-to-day behavior in that the flood stage today has high predictive power for the flood stage tomorrow. Assume that the flood stages are:

* Stage 0 (Normal): Average daily flow is below 90,000 $\mathrm{ft}^3$ (cubic feet per second = cfs). This is the normal river level.

* Stage 1 (Action Level): Average daily flow is between 90,000 cfs and 124,000 cfs.

* Stage 2 (Minor Flood): Average daily flow is between 124,000 cfs and 146,000 cfs.

* Stage 3 (Moderate Flood): Average daily flow is between 146,000 cfs and 170,000 cfs.

* Stage 4 (Extreme Flood): Average daily flow is above 170,000 cfs.

The table below shows the contents of an array with the probabilities of one stage transitioning into another stage from one day to the next.

In [4]:
data = [
    ["Stage 0 (Normal)", 0.9, 0.3, 0, 0, 0],
    ["Stage 1 (Action Level)", 0.05, 0.7, 0.4, 0, 0],
    ["Stage 2 (Minor Flood)", 0.025, 0, 0.6, 0.6, 0],
    ["Stage 3 (Moderate Flood)", 0.15, 0, 0, 0.4, 0.8],
    ["Stage 4 (Extreme Flood)", 0.01, 0, 0, 0, 0.2]
]
print(tabulate(data, headers=["Stage","Stage 0", "Stage 1", "Stage 2", "Stage 3", "Stage 4"],floatfmt=".3f"))

Stage                       Stage 0    Stage 1    Stage 2    Stage 3    Stage 4
------------------------  ---------  ---------  ---------  ---------  ---------
Stage 0 (Normal)              0.900      0.300      0.000      0.000      0.000
Stage 1 (Action Level)        0.050      0.700      0.400      0.000      0.000
Stage 2 (Minor Flood)         0.025      0.000      0.600      0.600      0.000
Stage 3 (Moderate Flood)      0.150      0.000      0.000      0.400      0.800
Stage 4 (Extreme Flood)       0.010      0.000      0.000      0.000      0.200


Mathematically, if $s_k$ is the state at day $k$ and $A$ is the array given in the table above then the difference equation  

$$\mathbf{s}_{k+1} = A\mathbf{s}_k$$

shows how a state will transition from day to day. For example, if we are currently in Stage 0 then $\mathbf{s}_0 = (1, 0, 0, 0, 0)$.
We can interpret this as “there is a probability of 1 that we are in Stage 0 today and there is a probability of 0 that we are in any other stage today.”

If we want to advance this model forward in time then we just need to iterate. In our example, the state tomorrow would be $\mathbf{s}_1 = A\mathbf{s}_0$. The state two days from now would be $\mathbf{s}_2 = A\mathbf{s}_1 = A(A\mathbf{s}_0) = A^2 \mathbf{s}_0$.

(a) Show that the state at day $n$ is $\mathbf{s}_n = A^n \mathbf{s}_0$.

In [5]:
A = np.zeros((5, 5))
for i in range(5):
  A[i][:] = data[i][1:]
print("A = \n", A)
s0 = np.array([1, 0, 0, 0, 0])
print("s_0 = ", s0)

A = 
 [[0.9   0.3   0.    0.    0.   ]
 [0.05  0.7   0.4   0.    0.   ]
 [0.025 0.    0.6   0.6   0.   ]
 [0.15  0.    0.    0.4   0.8  ]
 [0.01  0.    0.    0.    0.2  ]]
s_0 =  [1 0 0 0 0]


(b) If $n$ is large then the steady state solution to the difference equation in part (a) is given exactly by the power method iteration that we have studied in this chapter. Hence, as the iterations proceed they will be pulled toward the dominant eigenvector. Use the power method to find the dominant eigenvector of the matrix $A$.

In [6]:
xk = s0.copy()
for i in range(50):
  xk = A @ xk
  mu = np.amax(xk)
  xk = xk/mu
print(xk)

[1.         0.54170154 0.36593119 0.24041167 0.01159406]


(c) The vectors in this problem are called probability vectors in the sense that the vectors sum to 1 and every entry can be interpreted as a probability. Re-scale your answer from part (b) so that we can interpret the entries as probabilities. That is, ensure that the sum of the vector from part (b) is 1.

In [7]:
sn = xk/(np.linalg.norm(xk,1))
print(sn)
print(np.sum(sn))

[0.46304046 0.25082973 0.16944095 0.11132033 0.00536852]
1.0


(d) Interpret your answer to part (c) in the context of the problem. Be sure that your interpretation could be well understood by someone that does not know the mathematics that you just did.

*Ans:*

# Problem 2

*Modified Exercise 4.80: Error in LU for large random matrices*

For this problem we are going to run a numerical experiment to see how the process of solving the equation $A\mathbf{x}=\mathbf{b}$ using the $LU$ factorization performs on random coefficient matrices $A$ and random right-hand sides $\mathbf{b}$. We will compare against Python’s algorithm for solving linear systems.

(a) Create a loop that does the following:

1. Loop over the size of the matrix ($n$). 

2. Build a random matrix $A$ of size $n\times n$. You can do this with the code A = np.array( np.random.randn(n,n) )

3. Build a random vector $\mathbf{b}$ in $\mathbb{R}^n$. You can do this with the code b = np.array( np.random.randn(n,1) )

4. Find Python’s answer to the problem $A\mathbf{x}=\mathbf{b}$ using the command exact = np.linalg.solve(A,b)

5. Write code that uses your three $LU$ functions (myLU, lsolve, usolve) to find a solution to the equation $A\mathbf{x}=\mathbf{b}$.

6. Find the error between your answer and the exact answer using the code np.linalg.norm(x - exact)

(b) Make a plot (plt.semilogy()) that shows how the error behaves as the size of the problem changes. You should run this for matrices of larger and larger size but be warned that the loop will run for quite a long time if you go above $300 \times 300$ matrices. Just be patient.

(c) Conclusions: What do you notice in your final plot. What does this tell you about the behavior of our $LU$ decomposition code?

*Ans:*

## Problem 3

*Compute time for direct and iterative methods*

Compare the time required to implement the direct and iterative methods we've covered: built-in methods, PLU factorization, and Jacobi, Gauss-Seidel, and SOR iteration, using a tri-diagonal (strictly diagonally dominant) matrix $A$ and random vector $\mathbf{b}$.

For each method, determine the computational time required for implementation of each algorithm with the time class.

In [8]:
n = 10000
A = np.diag(3*np.ones(n))
A += np.diag(-1*np.ones(n-1), k=1)
A += np.diag(-1*np.ones(n-1), k=-1)
b = np.array(np.random.randn(n,1))
x0 = np.zeros(n)
numIter = 10
omega = 2

results = []

start = time()
soln = jacobi(A, b, x0, numIter)
computeTime = time() - start
results.append(["Jacobi", computeTime])

start = time()
soln = gauss_seidel(A, b, x0, numIter)
computeTime = time() - start
results.append(["Gauss-Seidel", computeTime])

start = time()
soln = lu_solve(lu_factor(A),b)
computeTime = time() - start
results.append(["PLU factorization", computeTime])

start = time()
soln = np.linalg.solve(A,b)
computeTime = time() - start
results.append(["Built-in solver", computeTime])

print(tabulate(results, headers=["Method", "Compute Time (s)"]))

Method               Compute Time (s)
-----------------  ------------------
Jacobi                        2.31379
Gauss-Seidel                  7.38973
PLU factorization            12.5296
Built-in solver              10.3168


## Problem 4

*Modified Exercise 4.86: Implementing the power method*

Find the largest eigenvalue and corresponding eigenvector of $A$ using the power method.

$$ A = \begin{bmatrix} 1 & 2 & 3 & 4 \\ 5 & 6 & 7 & 8 \\ 9 & 0 & 1 & 2 \\ 3 & 4 & 5 & 6 \end{bmatrix} $$

In [10]:
A = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6]).reshape((4,4))
xk = np.ones(4)

for i in range(10):
  xk = A @ xk
  mu = np.amax(xk)
  xk = xk/mu

print("Largest eigenvalue of A: ", mu)
print("Corresponding eigenvector of A: ", xk)

A1 = np.array([6, 5, 1, 2]).reshape((2, 2))
xk1 = np.ones(2)

for i in range(10):
  xk1 = A1 @ xk1
  mu1 = np.amax(xk1)
  xk1 = xk1/mu1

print("Largest eigenvalue of A1: ", mu1)
print("Corresponding eigenvector of A1: ", xk1)

Largest eigenvalue of A:  15.816775028485962
Corresponding eigenvector of A:  [0.3894048  1.         0.33030453 0.6947024 ]
Largest eigenvalue of A1:  7.000000059474238
Corresponding eigenvector of A1:  [1.  0.2]


## Problem 5

*Modified Exercise 4.65: Rate of convergence of the power method*

Recall that the power method relies on $A$ having a strictly diagonally domainant eigenvalue $\lambda_1$ such that $|\lambda_1|< |\lambda_2| \leqslant \cdots \leqslant |\lambda_n|$. Then

$$\lim_{k\to\infty} \frac{A^k\mathbf{x}}{\lambda_1^k}$$

converges to the dominant eigenvector. Estimate the speed of convergence of the power method by calculating the ratio $|\lambda_2/\lambda_1|$.

(a) Build a $4\times 4$ matrix $A$ with dominant eigenvalue $\lambda_1=1$ and all other eigenvalues less than 1 in absolute value. (Hint: To build a matrix with specific eigen-structure use the matrix factorization $A=PDP^{-1}$, where the columns of $P$ contain the eigenvectors of $A$ and the diagonal of $D$ contains the eigenvalues. In this case the $P$ matrix can be random but you need to control the $D$ matrix. Moreover, remember that $\lambda_3$ and $\lambda_4$ should be smaller than $\lambda_2$.)

In [12]:
e_vals = np.array([1, 0.75, 0.3, 0.2])
D = np.diag(e_vals)
P = np.array(np.random.randn(4, 4))
A = P @ D @ np.linalg.inv(P)
print(A)
np.linalg.eig(A)

[[ 2.58372541  0.97000647 -2.15530011  1.34768436]
 [-1.64164812 -0.16746462  2.02036268 -1.50400233]
 [ 0.6744252   0.40677532 -0.15384804  0.10213363]
 [-0.42956629 -0.16491945  0.36758246 -0.01241274]]


(array([1.  , 0.75, 0.3 , 0.2 ]),
 array([[-0.84755782,  0.05786788,  0.55765721,  0.40897979],
        [ 0.35441275,  0.89398837, -0.7800839 , -0.20483141],
        [-0.35514638,  0.44418026,  0.06750206,  0.70170404],
        [ 0.17294047, -0.01183255, -0.27555584,  0.54625184]]))

(b) Then choose several values of $\lambda_2$ and build an experiment to determine the number of iterations that it takes for the power method to converge to within a pre-determined tolerance to the dominant eigenvector. (Do this for at least five $\lambda_2$ values.)

(c) In the end you should produce a plot with the ratio $|\lambda_2/\lambda_1|$ on the horizontal axis and the number of iterations to converge to a fixed tolerance on the vertical axis. Discuss what you see in your plot.